# 구글

In [1]:
import os
import zipfile
import pandas as pd
from transformers import ElectraForSequenceClassification, AutoTokenizer, ElectraConfig
import torch

# 1. 모델 로드 및 초기화
zip_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base.zip'
extract_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base'

if not os.path.exists(extract_path):  # 이미 압축이 풀려있는지 확인
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

model_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base/saved_model'
config = ElectraConfig.from_pretrained(model_path)
model = ElectraForSequenceClassification.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 평가 모드로 설정

# 2. 감성 분석 함수 정의
def predict_sentiment(text):
    if not isinstance(text, str):
        return "알 수 없음"  # 문자열이 아닌 경우 기본값 반환
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "긍정적" if prediction == 1 else "부정적"

# 3. 입력/출력 폴더 설정
input_folder = "C:/fintech_service/final_project/data/리뷰추출/구글재분석"
output_folder = "C:/fintech_service/final_project/data/리뷰추출/구글재분석3"

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 4. 입력 폴더의 모든 CSV 파일에 대해 재분석 수행
csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

for csv_file in csv_files:
    input_path = os.path.join(input_folder, csv_file)
    # 파일 이름에 "_재분석3" 추가
    output_path = os.path.join(output_folder, f"{os.path.splitext(csv_file)[0]}_재분석3.csv")
    
    try:
        # CSV 파일 읽기
        df = pd.read_csv(input_path)
        
        # 문자열이 아닌 값(NaN 등)을 제거
        df = df.dropna(subset=['Review'])
        
        # 부정적으로 판별된 리뷰에 대해 재분석 (기존 컬럼 유지)
        if 'sentiment' in df.columns:
            df['Sentiment2'] = df.apply(lambda row: predict_sentiment(row['Review']), axis=1)
        else:
            print(f"'sentiment' 컬럼이 없습니다: {csv_file}")
            continue
        
        # 새롭게 전체 리뷰를 재분석하여 Sentiment3 컬럼에 추가
        df['Sentiment3'] = df['Review'].apply(predict_sentiment)

        # 결과 저장
        df.to_csv(output_path, index=False)
        print(f"파일 저장 완료: {output_path}")
    
    except Exception as e:
        print(f"파일 처리 중 오류 발생 ({csv_file}): {e}")

print("모든 파일 재분석 완료!")


파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\고가네철판불백_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\고래똥_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\고불초쌈밥_2호점_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\곰바로곰탕_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\곰작골나주곰탕_역삼초교점_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\교토일식_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\궁민김밥_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\깡돈_강남대로_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\낙여삼_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/구글재분석3\노브랜드버거_뱅뱅사거리점_reviews_google_분석_재분석_재분석3.csv
파일 저장 완료: C:

# 카카오

In [3]:
import os
import zipfile
import pandas as pd
from transformers import ElectraForSequenceClassification, AutoTokenizer, ElectraConfig
import torch
import numpy as np
import random

# 0. 시드 고정
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


# 1. 모델 로드 및 초기화
zip_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base.zip'
extract_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base'

if not os.path.exists(extract_path):  # 이미 압축이 풀려있는지 확인
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

model_path = 'C:/fintech_service/final_project/saved_model_KcELECTRA-base/saved_model'
config = ElectraConfig.from_pretrained(model_path)
model = ElectraForSequenceClassification.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()  # 평가 모드로 설정

# 2. 감성 분석 함수 정의
def predict_sentiment(text):
    if not isinstance(text, str):
        return "알 수 없음"  # 문자열이 아닌 경우 기본값 반환
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "긍정적" if prediction == 1 else "부정적"

# 3. 입력/출력 폴더 설정
input_folder = "C:/fintech_service/final_project/data/리뷰추출/카카오재분석"
output_folder = "C:/fintech_service/final_project/data/리뷰추출/카카오재분석3"

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 4. 입력 폴더의 모든 CSV 파일에 대해 재분석 수행
csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

for csv_file in csv_files:
    input_path = os.path.join(input_folder, csv_file)
    # 파일 이름에 "_재분석3" 추가
    output_path = os.path.join(output_folder, f"{os.path.splitext(csv_file)[0]}_재분석3.csv")
    
    try:
        # CSV 파일 읽기
        df = pd.read_csv(input_path)
        
        # 문자열이 아닌 값(NaN 등)을 제거
        df = df.dropna(subset=['Review'])
        
        # 부정적으로 판별된 리뷰에 대해 재분석 (기존 컬럼 유지)
        if 'sentiment' in df.columns:
            df['Sentiment2'] = df.apply(lambda row: predict_sentiment(row['Review']), axis=1)
        else:
            print(f"'sentiment' 컬럼이 없습니다: {csv_file}")
            continue
        
        # 새롭게 전체 리뷰를 재분석하여 Sentiment3 컬럼에 추가
        df['Sentiment3'] = df['Review'].apply(predict_sentiment)

        # 결과 저장
        df.to_csv(output_path, index=False)
        print(f"파일 저장 완료: {output_path}")
    
    except Exception as e:
        print(f"파일 처리 중 오류 발생 ({csv_file}): {e}")

print("모든 파일 재분석 완료!")


파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\9CAFE_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\고가네철판불백_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\고래똥_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\고불초쌈밥_2호점_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\고품격커피공장_뱅뱅사거리점_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\곰바로곰탕_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\곰작골나주곰탕_역삼초교점_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\교토일식_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\궁민김밥_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/fintech_service/final_project/data/리뷰추출/카카오재분석3\깡돈_reviews_kakao_분석_재분석_재분석3.csv
파일 저장 완료: C:/f